In [1]:
# Import pyTorch
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Tensorboard logger
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow
from cv2 import imshow as cv2_imshow
#glob for specific files importation
import glob

#to download and unzip
import wget

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# For Model saving and loading
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.events import EventStorage

# Show used device
device_num = torch.cuda.current_device()

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
import random

#Eval
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

print(torch.__version__)

import cv2 
print (cv2. __version__) 

1.7.1 True
gcc (i686-posix-dwarf-rev0, Built by MinGW-W64 project) 8.1.0
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

1.7.1
4.4.0


### Enregistrement de la dataset

In [2]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("obstacle_route", {}, "./coco_version/trainval.json", "./coco_version")

metadata

In [3]:
obstacle_route_metadata = MetadataCatalog.get("obstacle_route")
dataset_dicts = DatasetCatalog.get("obstacle_route")

WARNING [03/13 23:54:52 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/13 23:54:52 d2.data.datasets.coco]: Loaded 14 images in COCO format from ./coco_version/trainval.json


verify

In [4]:
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=obstacle_route_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow('Test',vis.get_image()[:, :, ::-1])
    cv2.waitKey(0); cv2.destroyAllWindows(); cv2.waitKey(1)

### train - MASK-RCNN

In [5]:
cfg = get_cfg()
cfg.merge_from_file("./../configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("obstacle_route",)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 0 ###
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = (
    20
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # 4 classes

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.DEVICE='cuda:0' # Gpu ou Cpu
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[03/13 23:55:07 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [03/13 23:55:07 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/13 23:55:07 d2.data.datasets.coco]: Loaded 14 images in COCO format from ./coco_version/trainval.json
[03/13 23:55:07 d2.data.build]: Removed 0 images with no usable annotations. 14 images left.
[03/13 23:55:07 d2.data.build]: Distribution of instances among all 5 categories:
|   category   | #instances   |  category  | #instances   |  category  | #instances   |
|:------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|     car      | 33           | motorcycle | 6            |   person   | 17           |
| traffic sign | 28           |   truck    | 11           |            |              |
|    total     | 95           |            |              |            |              |
[03/13 23:55:07 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (4, 256, 1, 1) 

[03/13 23:55:08 d2.engine.train_loop]: Starting training from iteration 0


C:\Users\G4nne\.conda\envs\Deep\lib\site-packages\detectron2\structures\masks.py:345: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  item = item.nonzero().squeeze(1).cpu().numpy().tolist()


[03/13 23:55:54 d2.utils.events]:  eta: 0:00:00  iter: 19  total_loss: 2.604  loss_cls: 1.049  loss_box_reg: 0.7545  loss_mask: 0.6794  loss_rpn_cls: 0.06473  loss_rpn_loc: 0.03888  time: 1.9972  data_time: 0.1804  lr: 0.00039962  max_mem: 2546M
[03/13 23:55:54 d2.engine.hooks]: Overall training speed: 18 iterations in 0:00:35 (1.9972 s / it)
[03/13 23:55:54 d2.engine.hooks]: Total training time: 0:00:39 (0:00:03 on hooks)


#### Evaluate Mask - RCNN

In [29]:
evaluator = COCOEvaluator("obstacle_route", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "obstacle_route")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

WARNING [03/06 23:51:10 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/06 23:51:10 d2.data.datasets.coco]: Loaded 14 images in COCO format from ./coco_version/trainval.json
[03/06 23:51:10 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[03/06 23:51:10 d2.data.common]: Serializing 14 elements to byte tensors and concatenating them all ...
[03/06 23:51:11 d2.data.common]: Serialized dataset takes 0.06 MiB
[03/06 23:51:11 d2.evaluation.evaluator]: Start inference on 14 images
[03/06 23:51:15 d2.evaluation.evaluator]: Inference done 11/14. 0.2516 s / img. ETA=0:00:01
[03/06 23:51:17 d2.utils.memory]: Attempting to copy inputs of <function paste_masks_in_image at 0x000001D464974430> to CPU due to CUDA OOM
[03/06 23:51:18 d2.evaluation.evaluator]: Total inference time: 0:00:05.862758 (0.651418 s / img per de

### train - Faster-RCNN

In [16]:
cfg = get_cfg()
cfg.merge_from_file("./../configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("obstacle_route",)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 0 ###
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = (
    20
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # 4 classes

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.DEVICE='cuda:0' # Gpu ou Cpu
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[03/07 22:18:11 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [03/07 22:18:11 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/07 22:18:11 d2.data.datasets.coco]: Loaded 14 images in COCO format from ./coco_version/trainval.json
[03/07 22:18:11 d2.data.build]: Removed 0 images with no usable annotations. 14 images left.
[03/07 22:18:11 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[03/07 22:18:11 d2.data.build]: Using training sampler TrainingSampler
[03/07 22:18:11 d2.data.common]: Serializing 14 elements to byte tensors and concatenating them all ...
[03/07 22:18:11 d2.data.common]: Serialized dataset takes 0.06 MiB


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.


[03/07 22:18:12 d2.engine.train_loop]: Starting training from iteration 0


C:\Users\G4nne\.conda\envs\Deep\lib\site-packages\detectron2\modeling\roi_heads\fast_rcnn.py:225: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


[03/07 22:18:36 d2.utils.events]:  eta: 0:00:00  iter: 19  total_loss: 2.05  loss_cls: 1.267  loss_box_reg: 0.8136  loss_rpn_cls: 0.05319  loss_rpn_loc: 0.04496  time: 0.9596  data_time: 0.1403  lr: 0.00039962  max_mem: 2913M
[03/07 22:18:36 d2.engine.hooks]: Overall training speed: 18 iterations in 0:00:17 (0.9596 s / it)
[03/07 22:18:36 d2.engine.hooks]: Total training time: 0:00:19 (0:00:01 on hooks)


#### Evaluate trained Faster - RCNN

In [17]:
evaluator = COCOEvaluator("obstacle_route", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "obstacle_route")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

WARNING [03/07 22:18:37 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/07 22:18:37 d2.data.datasets.coco]: Loaded 14 images in COCO format from ./coco_version/trainval.json
[03/07 22:18:37 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[03/07 22:18:37 d2.data.common]: Serializing 14 elements to byte tensors and concatenating them all ...
[03/07 22:18:37 d2.data.common]: Serialized dataset takes 0.06 MiB
[03/07 22:18:37 d2.evaluation.evaluator]: Start inference on 14 images
[03/07 22:18:39 d2.evaluation.evaluator]: Inference done 11/14. 0.1880 s / img. ETA=0:00:00
[03/07 22:18:40 d2.evaluation.evaluator]: Total inference time: 0:00:02.089396 (0.232155 s / img per device, on 1 devices)
[03/07 22:18:40 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:01 (0.189964 s / img per device, on 1 

TypeError: list indices must be integers or slices, not str

Initialisation d'une configuration de modèle, (à reconfigurer pour le kit de dev)

In [9]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("obstacle_route")
predictor = DefaultPredictor(cfg)

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (5, 1024) in the checkpoint but (81, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (5,) in the checkpoint but (81,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (16, 1024) in the checkpoint but (320, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (16,) in the checkpoint but (320,) in the model! You might want to double check if this is expected.


[03/07 22:15:14 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

FileNotFoundError: [Errno 2] No such file or directory: 'datasets\\coco/annotations/instances_train2017.json'

### Evaluate default Mask R-CNN

In [21]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE='cuda:0'
predictor = DefaultPredictor(cfg)
cfg.DATASETS.TRAIN = ("obstacle_route",)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

[03/07 22:25:13 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [03/07 22:25:13 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/07 22:25:13 d2.data.datasets.coco]: Loaded 14 images in COCO format from ./coco_version/trainval.json
[03/07 22:25:13 d2.data.build]: Removed 0 images with no usable annotations. 14 images left.
[03/07 22:25:13 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[03/07 22:25:13 d2.data.build]: Using training sampler TrainingSampler
[03/07 22:25:13 d2.data.common]: Serializing 14 elements to byte tensors and concatenating them all ...
[03/07 22:25:13 d2.data.common]: Serialized dataset takes 0.06 MiB


In [ ]:
evaluator = COCOEvaluator("obstacle_route", ("bbox", "segm"), False, output_dir="./output/Default_Mask")
val_loader = build_detection_test_loader(cfg, "obstacle_route")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

WARNING [03/07 22:25:18 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/07 22:25:18 d2.data.datasets.coco]: Loaded 14 images in COCO format from ./coco_version/trainval.json
[03/07 22:25:18 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[03/07 22:25:18 d2.data.common]: Serializing 14 elements to byte tensors and concatenating them all ...
[03/07 22:25:18 d2.data.common]: Serialized dataset takes 0.06 MiB
[03/07 22:25:18 d2.evaluation.evaluator]: Start inference on 14 images


### Test du modèle 

Importation d'une image aléatoire des images de tests

In [ ]:
# random_file = random.choice(glob.glob("./test/*.jpg"))
random_file = ("./test/image2.jpg")
random_image = cv2.imread(random_file)

Visualisation de l'image importée

In [70]:
cv2_imshow('Test1',random_image)  
cv2.waitKey(0); cv2.destroyAllWindows(); cv2.waitKey(1)

-1

Sauvegarde du modèle, copier la configuration reste plus facile vu que celle ci ne sera pas souvent modifiée

In [67]:
outputs = predictor(random_image)

# Objets détéctés et leurs masques
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

tensor([0], device='cuda:0')
Boxes(tensor([[161.8120,  66.0881, 228.7774, 114.2900]], device='cuda:0'))


Visualisation de l'image avec les prédictions

In [68]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(random_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow('Test2',out.get_image()[:, :, ::-1]) 
cv2.waitKey(0); cv2.destroyAllWindows(); cv2.waitKey(1)

-1